In [1]:
import torch
import os
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Laptop GPU'

In [3]:
data1=torch.load("../Data/cheating_Dataset.pth")
data2=torch.load("../Data/non_cheating_Dataset.pth")

C:\Users\abhis\AppData\Local\Temp\ipykernel_9160\1018567209.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data1=torch.load("../Data/cheating_Dataset.pth")
C:\Users\abh

In [4]:
print(data1.shape,data2.shape)

torch.Size([685, 20, 17, 3]) torch.Size([681, 20, 17, 3])


In [5]:
y1=torch.load("../Data/cheating_ylabel.pth")
y2=torch.load("../Data/non_cheating_ylabel.pth")

C:\Users\abhis\AppData\Local\Temp\ipykernel_9160\2512688000.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  y1=torch.load("../Data/cheating_ylabel.pth")
C:\Users\abhis\A

In [6]:
print(y1.shape,y2.shape)

torch.Size([685]) torch.Size([681])


In [7]:
data=torch.cat((data1,data2),dim=0)
data.shape

torch.Size([1366, 20, 17, 3])

In [8]:
y=torch.cat((y1,y2),dim=0)
y.shape

torch.Size([1366])

In [9]:
data=data.reshape(data.shape[0],data.shape[1],-1)
data.shape

torch.Size([1366, 20, 51])

In [17]:
y=y-1

<h3>Load Data</h3>

<h3>Model Architecture</h3>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
# Define the model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, 64, batch_first=True)
        self.lstm2 = nn.LSTM(64, 128, batch_first=True)
        self.lstm3 = nn.LSTM(128, 64, batch_first=True)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.relu(self.fc1(x[:, -1, :]))  # Use the last time step's output
        x = self.relu(self.fc2(x))
        #x = self.softmax(self.fc3(x))
        x = self.fc3(x)
        return x

In [19]:
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(data, y)
dataloader = DataLoader(dataset, batch_size=60, shuffle=True)


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [21]:
input_size = data.shape[2]  # 132
num_classes = 2  # 2
model = LSTMModel(input_size, num_classes).to(device)
model.to(device)

LSTMModel(
  (lstm1): LSTM(51, 64, batch_first=True)
  (lstm2): LSTM(64, 128, batch_first=True)
  (lstm3): LSTM(128, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
print("Model initialized and moved to device:", device)

Model initialized and moved to device: cuda


In [23]:
num_epochs = 30 

for epoch in range(num_epochs):
    model.train()  
    epoch_loss = 0.0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

      
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)  
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.4f}")


Epoch 1/30, Loss: 0.7028
Epoch 2/30, Loss: 0.6921
Epoch 3/30, Loss: 0.6960
Epoch 4/30, Loss: 0.6880
Epoch 5/30, Loss: 0.4271
Epoch 6/30, Loss: 0.3151
Epoch 7/30, Loss: 0.2028
Epoch 8/30, Loss: 0.2290
Epoch 9/30, Loss: 0.1999
Epoch 10/30, Loss: 0.1698
Epoch 11/30, Loss: 0.1744
Epoch 12/30, Loss: 0.1791
Epoch 13/30, Loss: 0.1275
Epoch 14/30, Loss: 0.1196
Epoch 15/30, Loss: 0.1096
Epoch 16/30, Loss: 0.1155
Epoch 17/30, Loss: 0.1077
Epoch 18/30, Loss: 0.0987
Epoch 19/30, Loss: 0.0936
Epoch 20/30, Loss: 0.0972
Epoch 21/30, Loss: 0.0987
Epoch 22/30, Loss: 0.1662
Epoch 23/30, Loss: 0.0990
Epoch 24/30, Loss: 0.0923
Epoch 25/30, Loss: 0.1040
Epoch 26/30, Loss: 0.0843
Epoch 27/30, Loss: 0.0919
Epoch 28/30, Loss: 0.0832
Epoch 29/30, Loss: 0.0826
Epoch 30/30, Loss: 0.0879


In [24]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        outputs = model(batch_x)
        predictions = torch.argmax(outputs, dim=1)
        label = batch_y
        total_correct += (predictions == label).sum().item()
        total_samples += label.size(0)

    print(f"Accuracy: {total_correct / total_samples:.4f}")


Accuracy: 0.9575


In [25]:
torch.save(model, "../models/model.pth")

In [26]:
loaded_model = torch.load("../models/model.pth")
loaded_model.to(device)  # Move to the appropriate device if needed
loaded_model.eval()  # Set the model to evaluation mode


C:\Users\abhis\AppData\Local\Temp\ipykernel_9160\1228541698.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load("../models/model.pth")


LSTMModel(
  (lstm1): LSTM(51, 64, batch_first=True)
  (lstm2): LSTM(64, 128, batch_first=True)
  (lstm3): LSTM(128, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [27]:
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = loaded_model(batch_x)
        predictions = torch.argmax(outputs, dim=1)
        label = batch_y
        total_correct += (predictions == label).sum().item()
        total_samples += label.size(0)

    print(f"Accuracy: {total_correct / total_samples:.4f}")

Accuracy: 0.9575


In [28]:
outputs = loaded_model(batch_x)
predictions = torch.argmax(outputs, dim=1)

In [29]:
input_sequence = data[800]  

# Convert NumPy array to PyTorch tensor
input_tensor = torch.tensor(input_sequence, dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension: shape becomes (1, 20, 132)

# Move to the appropriate device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)
loaded_model.to(device)

# Perform prediction
with torch.no_grad():
    output = loaded_model(input_tensor)  # Output shape: (1, num_classes)
    print(output)
    prediction = torch.argmax(output, dim=1)  # Get the predicted class index

# Print the prediction
print(f"Predicted class: {prediction.item()}")

tensor([[-1.5230,  2.2313]], device='cuda:0')
Predicted class: 1


C:\Users\abhis\AppData\Local\Temp\ipykernel_9160\2319691819.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(input_sequence, dtype=torch.float32)


additional softmax layer in model is not nessery because of crossentropy already applays softmax automaticall
no need to add extra one if added reduces probablity

label is converted to 1d during traning and only index value is taken because of how crossentropy works it expects the 
label to be size(1,batch size) 2dim has index value of class 